In [ ]:
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo(id="QjzMFdrVeF4", width="100%")



In [ ]:

import pandas as pd


In [ ]:
import networkx as nx

In [ ]:
G = nx.powerlaw_cluster_graph(50, 1, 0.1)

In [ ]:
from email.utils import getaddresses, parseaddr
import mailbox

def mbox_graph(mboxname):
    mbox = mailbox.mbox(mboxname)  # parse unix mailbox

    G = nx.MultiDiGraph()  # create empty graph

    # parse each messages and build graph
    for msg in mbox:  # msg is python email.Message.Message object
        (source_name, source_addr) = parseaddr(msg["From"])  # sender
        # get all recipients
        # see https://docs.python.org/3/library/email.html
        tos = msg.get_all("to", [])
        ccs = msg.get_all("cc", [])
        resent_tos = msg.get_all("resent-to", [])
        resent_ccs = msg.get_all("resent-cc", [])
        all_recipients = getaddresses(tos + ccs + resent_tos + resent_ccs)
        # now add the edges for this mail message
        for (target_name, target_addr) in all_recipients:
            G.add_edge(source_addr, target_addr, message=msg)

    return G



In [ ]:
# In order to check a directed Multigraph uncomment the line below
# Warning... some of the functions applied in the unconnected graphs arent available (or dont make sense!).expect less functionality available

#G = mbox_graph("202012.mbox")

In [ ]:

pos = nx.kamada_kawai_layout(G)
nx.draw(G, pos, node_color= ["black"],node_size=100)

In [ ]:
from nxviz import MatrixPlot

m = MatrixPlot(G)
m.draw()

In [ ]:
from nxviz import CircosPlot

c = CircosPlot(G,)
c.draw()

In [ ]:
nx.number_connected_components(G)

In [ ]:
G_comp  = [G.subgraph(c).copy() for c in nx.connected_components(G)]

In [ ]:
from networkx.algorithms import *

In [ ]:
hc = harmonic_centrality(G)


#The closeness centrality (or closeness) of a node is the average length of the shortest path between the node 
#and all other nodes in the graph. 
#Thus the more central a node is, the closer it is to all other nodes.

cc=closeness_centrality(G)


#Degree centrality is the number of links incident upon a node (i.e., the number of ties that a node has)
dc=degree_centrality(G)

#Betweenness centrality quantifies the number of times a node acts as a bridge along the shortest path 
#between two other nodes. 
bc=betweenness_centrality(G)





hc = harmonic_centrality(G)


In [ ]:
def get_top_keys(dictionary, top):
    items = dictionary.items()
    items = sorted(items,reverse=True, key=lambda x: x[1])
    return list(map(lambda x: x[0], items[:top]))

In [ ]:
import pandas as pd

In [ ]:
nodes = list(G.nodes)
counts = nx.number_connected_components(G)
output = []
for i, node in enumerate(nodes):
    subG = G.subgraph(nodes[:i]+nodes[i+1:])
    subGCount = nx.number_connected_components(subG)
    if subGCount>counts:
        output.append((node,1))
    else:
        output.append((node,0))

articulationdf= pd.DataFrame(output,columns=["nodeid","articulation"])
artdf=articulationdf.query("articulation==1")

In [ ]:
articulation_color_map=[]
for node in G:
    #print(node)
    if node in artdf.nodeid.values:
        articulation_color_map.append('red')
    else: 
        articulation_color_map.append('yellow')  

In [ ]:





pos = nx.kamada_kawai_layout(G,scale=2,dim=2)
nx.draw(G, pos, node_color= articulation_color_map,edge_color='b',node_size=100)


In [ ]:


closenodes = get_top_keys(cc,10)

closeness_color_map=[]
for node in G:
    #print(node)
    if node in closenodes:
        closeness_color_map.append('green')
    else: 
        closeness_color_map.append('yellow')  

In [ ]:
nx.draw(G, pos, node_color= closeness_color_map,edge_color='b',node_size=100)

In [ ]:

btwnodes = get_top_keys(cc,10)

btwn_color_map=[]
for node in G:
    #print(node)
    if node in btwnodes:
        btwn_color_map.append('cyan')
    else: 
        btwn_color_map.append('yellow')  
        
nx.draw(G, pos, node_color= btwn_color_map,edge_color='b',node_size=100)

In [ ]:
def avg_neigh_degree(g):
    avgndeg = {}
    for n in g.nodes():
        if g.degree(n):
            avgndeg[n] = float(sum(g.degree(i) for i in g[n]))/g.degree(n)
    return avgndeg

In [ ]:

avgnnodes = get_top_keys(avg_neigh_degree(G),10) 
avgn_color_map=[]
for node in G:
    #print(node)
    if node in avgnnodes:
        avgn_color_map.append('black')
    else: 
        avgn_color_map.append('yellow')  
        
nx.draw(G, pos, node_color= avgn_color_map,edge_color='b',node_size=100)

In [ ]:
pd.DataFrame.from_dict(bc,orient="index",columns=["btwn_centrality"]).\
reset_index().rename(columns={"index": "node", "btwn_centrality": "btwn_centrality"})